In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
import pickle
import numpy as np


In [2]:
# Обучим модель линейной регрессии на встроенном датасете Diabetes dataset.
# Опустим кросс-валидацию и разделение выборки на обучение и тест, так как для наших целей это не важно.
X, y = load_diabetes(return_X_y=True)
regressor = LinearRegression()
regressor.fit(X, y)


LinearRegression()

In [6]:
# Импортируем модуль и воспользуемся методом dumps:
model = pickle.dumps(regressor)
print(type(model))
print(type(regressor))


<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


In [7]:
# Восстановим объект Python:
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

LinearRegression()

In [14]:
# Сохраним объект прямо в файл:
with open("./prod_2.pkl", "wb") as output:
    pickle.dump(regressor, output)


In [15]:
# Файл prod_1.pkl так же легко десериализовать:
with open("./prod_2.pkl", "rb") as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file

LinearRegression()

In [11]:
# Убедимся, что методы и результаты предсказаний обученной модели и загруженной из файла совпадают:
print(all(regressor.predict(X) == regressor_from_bytes.predict(X)))
print(all(regressor.predict(X) == regressor_from_file.predict(X)))


True
True


In [12]:
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from joblib import dump, load


X, y = load_diabetes(return_X_y=True)

pipe = Pipeline(
    [
        ("Scaling", MinMaxScaler()),
        ("FeatureSelection", SelectKBest(f_regression, k=5)),
        ("Linear", LinearRegression()),
    ]
)


pipe.fit(X, y)
pipe.predict(X[1:2])
s1 = pickle.dumps(pipe)
pipe_from_bytes = pickle.loads(s1)

print(all(pipe.predict(X) == pipe_from_bytes.predict(X)))


True


Обратите внимание, что для работы с файлами используются методы pickle.dump и pickle.load, а для работы с сериализованными объектами pickle.dumps и pickle.loads.

Как мы видим, Pickle прекрасно справляется со своей задачей, однако иногда массивы данных бывают настолько большими, что после загрузки из Pickle можно не восстановить объект полностью. 

В таких случаях лучше использовать Joblib вместо Pickle. Этот модуль более эффективен для объектов, которые содержат большие массивы данных. Он оптимизирован для быстрой и надежной работы с данными большого объема. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой.

Для иллюстрации работы сохраним обученную модель:



In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
regressor = LinearRegression()
regressor.fit(X, y)

from joblib import dump, load

dump(regressor, "prod_2.joblib")

clf_from_jobliv = load("prod_2.joblib")
all(regressor.predict(X) == clf_from_jobliv.predict(X))

True

## 8.1

Коллега обучил модель и теперь просит вас проверить её на ваших данных. Он присылает вам Pickle-файл. Загрузите модель, используя модуль Pickle. 



In [17]:
with open("hw1.pkl", "rb") as hw_file:
    model = pickle.load(hw_file)

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


/Users/alexigna/projects/skillfactory/data science/skillfactory_rds/venv/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
type(model)

sklearn.linear_model._base.LinearRegression

In [29]:
X_test = np.array([1, 1, 1, 0.661212487096872])
y_pred = model.predict(X_test.reshape(1, -1))
y_pred

array([0.666])

In [36]:
# У модели есть два поля с именами a и b. Создайте из них словарь с такими же
# именами ключей и значениями, а затем сохраните в файл с помощью модуля Pickle. 
dict_to_file = {
    "a": model.a,
    "b": model.b,
}
with open("prod_2.8.4.pkl", "wb") as pickle_file:
    pickle.dump(dict_to_file, pickle_file)
    


## 9

In [37]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()["feature_names"]

scaler = MinMaxScaler()
pipe = Pipeline(
    [
        ("Scaling", MinMaxScaler()),
        ("Linear", LinearRegression()),
    ]
)
# Тренировка пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipe.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="prod_2.9.pipe.pmml")


### 9.6

In [ ]:
import onnxruntime as rt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(X_test)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в onnx формат
initial_type = [('float_input', FloatTensorType([None, 13]))]
model_onnx = convert_sklearn(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через onnxruntime
sess = rt.InferenceSession("model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx) 